<a href="https://colab.research.google.com/github/zxarda01/Tweet-Sentiment-Classification-/blob/main/SVM_Word2Vec_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Data Preparation
!wget https://raw.githubusercontent.com/whyjay17/Twitter-Sentiment-Analysis/master/data/twitter-2016dev-A.txt


--2021-01-02 03:03:36--  https://raw.githubusercontent.com/whyjay17/Twitter-Sentiment-Analysis/master/data/twitter-2016dev-A.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 294054 (287K) [text/plain]
Saving to: ‘twitter-2016dev-A.txt’

twitter-2016dev-A.t 100%[===================>] 287.16K  --.-KB/s    in 0.04s   

2021-01-02 03:03:36 (7.33 MB/s) - ‘twitter-2016dev-A.txt’ saved [294054/294054]



In [2]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-01-02 03:03:38--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.82.70
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.82.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  73.1MB/s    in 24s     

2021-01-02 03:04:02 (64.6 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [4]:
# Package Installation
!pip install scikit-learn
!pip install nltk
!pip install numpy
!pip install gensim

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Data Loading
import csv
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from gensim.models.keyedvectors import KeyedVectors

tweet_data_path = '/content/twitter-2016dev-A.txt'
word2vec_path = '/content/GoogleNews-vectors-negative300.bin.gz'
tweet_tokenizer = TweetTokenizer()

with open(tweet_data_path , encoding='utf-8') as f:
    reader = csv.reader(f, delimiter="\t")
    tweet_data = list(reader)


print('[input]# of records:',len(tweet_data))
print('[input]sample data:',tweet_data[0])

[input]# of records: 1966
[input]sample data: ['638060586258038784', 'neutral', '05 Beat it - Michael Jackson - Thriller (25th Anniversary Edition) [HD] http://t.co/A4K2B86PBv']


In [6]:
# Data Preprocessing
tweet_data_parsed = []
stop = set(stopwords.words('english'))
for info in tweet_data:
    l = " ".join(tweet_tokenizer.tokenize(info[2].lower())).split(" ")
    filtered_sentence = [w for w in l if not w in stop and not w in string.punctuation
                         and ( w[0] != '@' and w[0] != '#' and w[:4] != 'http' )]
    tweet_data_parsed.append(filtered_sentence)

print('[parsed]# of records:',len(tweet_data_parsed))
print('[parsed]sample data:',tweet_data_parsed[0])

# Label Data
tweet_class = np.zeros(len(tweet_data))
for i in range(len(tweet_data)):
    if tweet_data[i][1] == 'negative':
        tweet_class[i] = 0
    elif tweet_data[i][1] == 'neutral':
        tweet_class[i] = 1
    elif tweet_data[i][1] == 'positive':
        tweet_class[i] = 2

print('[class]# of records:',len(tweet_class))
print('[class]sample data:',tweet_class[0])

[parsed]# of records: 1966
[parsed]sample data: ['05', 'beat', 'michael', 'jackson', 'thriller', '25th', 'anniversary', 'edition', 'hd']
[class]# of records: 1966
[class]sample data: 1.0


In [7]:
print('Model Loading Started')
model = KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
print('Model Loading Finished')

Model Loading Started
Model Loading Finished


In [8]:
# Feature(word2vec) 
tweet_word2vec = []
# adds the word2vec average
for tweet in tweet_data_parsed:
    average_vec = np.zeros(300)
    for word in tweet:
        if word in model.wv:
            average_vec += (model.wv[word] / len(tweet))
        else:
            pass
    tweet_word2vec.append(average_vec)

# train test split
ratio = len(tweet_data) * 0.2
tweet_train = tweet_data_parsed[: -int(ratio)]
tweet_test = tweet_data_parsed[-int(ratio):]

x_train = tweet_word2vec[: -int(ratio)]
y_train = tweet_class[: -int(ratio)]

x_test = tweet_word2vec[-int(ratio):]
y_test = tweet_class[-int(ratio):]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [22]:
import time
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV 
from sklearn import preprocessing

x_train = preprocessing.scale(x_train)


print("started") 
start = time.time()

param_grid = {'C': [0.1,1,10] } 
clf = GridSearchCV(LinearSVC(max_iter = 50000), param_grid, refit = True, verbose = 3) 
clf.fit(x_train,y_train)

end = time.time()
print("finished")
print("duration", end-start)
result1 = clf.predict(x_test)

started
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] C=0.1 ...........................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............................... C=0.1, score=0.498, total=   4.3s
[CV] C=0.1 ...........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s remaining:    0.0s


[CV] ............................... C=0.1, score=0.416, total=   3.8s
[CV] C=0.1 ...........................................................


...

[CV] ................................ C=10, score=0.419, total= 1.7min
[CV] C=10 ............................................................


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[CV] ................................ C=10, score=0.395, total= 1.7min
[CV] C=10 ............................................................


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 12.9min finished


[CV] ................................ C=10, score=0.427, total= 1.8min
finished
duration 780.9675941467285


In [25]:
import numpy as np
from sklearn.metrics import confusion_matrix

def cal_precision_recall(y, pred) :

    cnf_matrix = confusion_matrix(y, pred)
    print(cnf_matrix)

    recall = np.diag(cnf_matrix) / np.sum(cnf_matrix, axis = 1)
    precision = np.diag(cnf_matrix) / np.sum(cnf_matrix, axis = 0)

    avg_recall = np.mean(recall)
    avg_precision = np.mean(precision)

    print("avg_recall",avg_recall)
    print("avg_precision",avg_precision)
pred = clf.predict(x_test)
cal_precision_recall(y_test,pred)

[[  0  30  41]
 [  0  51  99]
 [  0  19 153]]
avg_recall 0.4098449612403101
avg_precision nan


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
